# Author: Young Beum Cho
# Kroll Lab, Department of Developmental Biology
# Program for gene list + keyword searching on ncbi/gds

In [44]:
import docx
import pandas as pd
import numpy as np
from docx import Document
import selenium
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.common.exceptions import NoSuchElementException 
import numpy as np
from numpy import nan
import bs4
import requests
import time
import pandas as pd
import re
from selenium.webdriver.common.by import By

# Execute above and below cells first
* To change the keyword search targets, modify search_element function
# First-level functions

The code below contains several first-level functions that serve different purposes in the web scraping process. These functions include:

- `search_element(element)`: This function constructs the search query to be used in the search box.
- `open_geo()`: This function opens the Geo website using a Chrome driver.
- `set_box(dr)`: This function finds and sets the search box element.
- `set_btn(dr)`: This function finds and sets the search button element.
- `set_next1(dr)`: This function finds and sets the next button element (if it exists).
- `set_next2(dr)`: This function finds and sets the next button element (if it exists).
- `check_exists_by_xpath(xpath, dr)`: This function checks if an element exists based on the given XPath.
- `type_click(str, box, btn)`: This function types the search query and clicks the search button.
- `click_next(next_btn)`: This function clicks the next button.
- `clearBox(box)`: This function clears the search box.
- `one_page_all_title(page)`: This function retrieves all titles appearing on the searched page.
- `one_page_all_url(page)`: This function retrieves all URLs linked to each individual search result.
- `page_num(page)`: This function retrieves the current page number and the last page number.
- `get_response(url)`: This function retrieves the response from a given URL.

Please note that this Markdown cell provides a brief overview of the functions included in the code. For more details on each function's implementation and usage, please refer to the code itself.


In [7]:
#first-level functions

#what will go into the search box with elements as every genes
def search_element(element):
    return r'"' + element + r'" & ' + r'"' + "homo sapiens" + r'" & "' + "Genome binding/occupancy profiling by high throughput sequencing" + r'"'

#open url
def open_geo():
    #open geo website
    driver = Chrome("chromedriver.exe")
    time.sleep(1)
    url = 'https://www.ncbi.nlm.nih.gov/gds/'
    driver.get(url)
    time.sleep(1)
    return driver

#find and set search box   
def set_box(dr):
    driver = dr
    search_box_path = '//*[@id="term"]'
    #set search box as variable
    search_box = driver.find_element("xpath", search_box_path)
    return search_box

#find and set search button
def set_btn(dr):
    driver = dr
    search_btn_path = '//*[@id="search"]'
    #set search box as variable
    search_btn = driver.find_element("xpath",search_btn_path)
    return search_btn

#find and set next button
def set_next1(dr):
    next_btn_path = '//*[@id="EntrezSystem2.PEntrez.Gds.Gds_ResultsPanel.Entrez_Pager.Page"]'
    #check if next button exists
    status = check_exists_by_xpath(next_btn_path, dr)
    if(status==1):
        next_btn = driver.find_element("xpath",next_btn_path)
        return next_btn
    else:
        return 0
    
#find and set next button
def set_next2(dr):
    next_btn_path = '/html/body/div[1]/div[1]/form/div[1]/div[4]/div/div[3]/div[2]/a[3]'
    #check if next button exists
    status = check_exists_by_xpath(next_btn_path, dr)
    if(status==1):
        next_btn = driver.find_element("xpath",next_btn_path)
        return next_btn
    else:
        return 0
        
def check_exists_by_xpath(xpath, dr):
    driver = dr
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return 0
    return 1

#type and click
def type_click(str, box, btn):
    search_box = box
    search_btn = btn
    search_box.send_keys(search_element(str))
    search_btn.click()
    
def click_next(next_btn):
    next_btn = next_btn
    next_btn.click()
    
#clear searchbox
def clearBox(box):
    search_box = box
    search_box.clear();

#gather all titles appearing in the page searched
def one_page_all_title(page):
    title_box = []
    for single in page.find_all('p', {'class' : 'title'}):
        title = single.find('a').text
        title_box.append(title)
    return title_box

#gather all urls linked to each individual search results
def one_page_all_url(page):
    link_box = []
    for single in page.find_all('p', {'class' : 'title'}):
        url = single.find_all('a')[0]
        url = url['href']
        link_box.append(url)
    return link_box

def page_num(page):
    single = page.find('h3', {'class' : 'page'})
    single = single.find('input', {'class' : 'num'})
    print(single)
    return int(single['value']), int(single['last'])
    

def get_response(url):
        response = requests.get(url).txt
        return reponse
    

    
    
    
    
    
    

# Execute by order ↓

# Second-level functions

The code below contains second-level functions that are used to find URLs for each of the titles retrieved from a page. These functions include:

- `one_page_summary(page, dp, test, i)`: This function takes a page, a DataFrame `dp`, a list `test`, and an index `i`. It retrieves the titles and URLs from the page, prepends the URLs with a front URL, and creates a new DataFrame `dp2` with the gene, title, and URL information. The function then concatenates `dp` and `dp2` and returns the resulting DataFrame.
- `prepend(ls, form)`: This function takes a list `ls` and a string `form`. It prepends each element in the list with the specified format `form` and returns the modified list.

These functions are used to gather information from the retrieved page, format the URLs, and create a new DataFrame with the gene, title, and URL information. The resulting DataFrame is then concatenated with the original DataFrame.

Please note that this Markdown cell provides a brief overview of the functions included in the code. For more details on each function's implementation and usage, please refer to the code itself.


In [25]:
#second-level functions
#find urls for each of the titles
def one_page_summary(page, dp, test, i):
    title_list = one_page_all_title(page)
    url_list = one_page_all_url(page)
    front_url = 'https://ncbi.nlm.nih.gov'
    
    url_list = prepend(url_list, front_url)
    
    current_gene = [test[i]] * len(url_list) #test is used here again
    all_list = [current_gene, title_list, url_list]

    dp2 = pd.DataFrame({
        'Gene': current_gene,
        'Title': title_list,
        'Link':  url_list
    })

    return pd.concat([dp, dp2], ignore_index=True)

def prepend(ls, form):
 
    # Using format()
    form += '{0}'
    ls = [form.format(i) for i in ls]
    return ls


# 'TF3.xlsx' must be present in the directory in which this code is located. Otherwise, need to be modified to fit the needs. 
# In the excel file which gene list will be imported, gene = df.iloc[:,0].tolist() means gene list needs to be at the first column

# Data Retrieval from GEO Website

The code below demonstrates the process of retrieving data from the GEO website using Selenium and BeautifulSoup. Here's an overview of the code:

1. Reading the input data: The code reads an Excel file ('TF3.xlsx') into a DataFrame called `df`. It then extracts the gene names from the first column of the DataFrame.

2. Initializing variables: The code initializes variables including `test` (used for test searches), `dp` (an empty DataFrame to store the results), and `driver` (the web driver for Selenium).

3. Opening the GEO website: The code opens the GEO website using the `open_geo()` function. It sets the search box and search button variables using the `set_box()` and `set_btn()` functions.

4. Performing searches: The code loops through each gene in the `test` list. For each gene, it types the gene name into the search box and clicks the search button using the `type_click()` function.

5. Handling search results: The code checks if there are any search results. If there are no results, it adds a row with NaN values to the DataFrame `dp`. If there are results, it retrieves the titles and URLs from the current page using the `one_page_all_title()` and `one_page_all_url()` functions. It then calls the `one_page_summary()` function to add the gene, title, and URL information to the DataFrame `dp`.

6. Handling multiple pages: If there are multiple pages of search results, the code navigates through the pages using the next button. It clicks the next button, updates the search box and search button variables, retrieves the new page source, and repeats the process of retrieving titles and URLs from the page using the `one_page_all_title()` and `one_page_all_url()` functions.

7. Saving the results: The code saves the resulting DataFrame `dp` to an Excel file named 'test.xlsx' using the `to_excel()` function.

8. Generating distinct values: The code generates a list `distinct_values` that contains the number of distinct values for each column in the DataFrame `dp`.

Please note that this Markdown cell provides a brief overview of the code. For more details on each function's implementation and usage, please refer to the code itself.



In [13]:
#open list of genes and save it as list
df = pd.read_excel('TF3.xlsx')
gene = df.iloc[:,0].tolist()
print(gene)

['HES4', 'NKX2-1', 'SOX3', 'POU3F1', 'HES1']


# After below code is executed, a pop-up window will appear and program will start to run. To abort, press ■ in the menubar 

In [26]:
#test for searches
test = gene 
dp = pd.DataFrame(columns = ['Gene', 'Title', 'Link'])

#open geo website
driver = open_geo()
box = set_box(driver)
btn = set_btn(driver)


for i in range(0, len(test)):
    type_click(test[i], box, btn) #change [test] to [gene] for search over entire genes
    #repeat below 5 everytime page changes/refreshes
    box = set_box(driver)
    btn = set_btn(driver)
    next_btn = set_next1(driver) #either the button exists or next_btn is 0
    src = driver.page_source
    page = bs4.BeautifulSoup(src)
    
    
    #find the number of titles current page. number of titles = number of search results 
    title_list = one_page_all_title(page)
    #find number of total pages,
    n_of_titles = len(title_list)   
    clearBox(box)     
    
    #if no search results, go next
    if(n_of_titles == 0):
        
        dp2 = pd.DataFrame({
            'Gene': [test[i]],
            'Title': [np.nan],
            'Link':  [np.nan]
        })
        dp = pd.concat([dp, dp2], ignore_index=True)
        
        
    else:     
        #if # of search results <= 20
        if(next_btn == 0):
            dp = one_page_summary(page, dp, test, i)
        else:
            current, last = page_num(page)
            while current <= last:
                if current == last:
                        dp = one_page_summary(page, dp, test, i)
                else:
                        dp = one_page_summary(page, dp, test, i)
                        click_next(next_btn)
                        box = set_box(driver)
                        btn = set_btn(driver)
                        next_btn = set_next2(driver) #either the button exists or next_btn is 0
                        src = driver.page_source
                        page = bs4.BeautifulSoup(src)
                current += 1

    time.sleep(1)
    

dp.to_excel('test.xlsx', index = False, encoding = 'utf8')
gene_output = dp.iloc[:,0].tolist()
                    
                    
                    
                
        
    
    

    
    
    
    
    



C:\Users\choyo\AppData\Local\Temp\ipykernel_12472\2777344704.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome("chromedriver.exe")


In [29]:
distinct_values = []
for col in dp.columns:
    #dinstincts
    distinct_values.append(len(dp[col].unique()))

In [30]:
distinct_values

[5, 40, 40]

## Part2

### Data Processing from Retrieved Results

The code below demonstrates the process of processing the retrieved results from the previously saved Excel file ('test.xlsx'). Here's an overview of the code:

1. Reading the retrieved results: The code reads the Excel file ('test.xlsx') into a DataFrame called `df`. It extracts the URLs and gene names from the DataFrame.

2. Processing the URLs: The code retrieves the URLs from the DataFrame using the `iloc` function and stores them in a list called `urls`. It also retrieves the first URL from the list and assigns it to a variable `urls[0]`.

3. Processing the gene names: The code retrieves the gene names from the DataFrame using the `iloc` function and stores them in a list called `genes`.

Please note that this Markdown cell provides a brief overview of the code. For more details on each step's implementation and usage, please refer to the code itself.


In [31]:
#open list of genes and save it as list
df = pd.read_excel('test.xlsx')
urls = df.iloc[:,2].tolist()
urls[0]
genes = df.iloc[:,0].tolist()

# Data Processing from Retrieved Results (Part 2)

The code below continues the data processing from the retrieved results. It includes several functions for further processing and analysis. Here's an overview of the functions:

1. `open_url(url, dr)`: This function opens a given URL using the provided driver (`dr`). It checks if the URL is not null before opening it.

2. `find_journal(page)`: This function searches for the journal information on the given page and returns the journal name. If the journal information is not found, it returns NaN.

3. `find_organization(page)`: This function searches for the organization name on the given page and returns the organization name. If the organization information is not found, it returns NaN.

4. `second_largest(ls)`: This function finds the second-largest value in a list (`ls`) and returns it.

5. `set_more(dr)`: This function finds and sets the "More" button on the page. It checks for the existence of the button at different positions and returns the appropriate button element.

6. `qc_status(page)`: This function calculates the quality control status based on the presence of "PassedQC" and "FailedQC" strings on the page. It returns the percentage of passed QC and failed QC.

7. `gdv_checker(page, dr)`: This function checks for the existence of the "gdv_button" on the page using the provided driver (`dr`). It returns a status indicating its existence.

8. `count_gene(page, gene, i)`: This function counts the occurrences of a specific gene (`gene[i]`) on the page. It returns a string with all the gene names and the total count.

Please note that this Markdown cell provides a brief overview of the code. For more details on each function's implementation and usage, please refer to the code itself.


In [46]:
def open_url(url, dr):
    if(pd.isnull(url) == False):
        dr.get(url)
        return 1
    else: 
        return 0

#gather all titles appearing in the page searched
def find_journal(page):
    try:
        jrl = page.find("span", "source")
        return jrl.text
    except AttributeError:
        return np.nan
    
def find_organization(page):
    try:
        org = page.find("td", text="Organization name").find_next_sibling("td")
        return org.text
    except AttributeError:
        return np.nan
    
def second_largest(ls):
    ls.sort()
    return ls[-2]

#find and set more button
def set_more(dr):
    all_more = []
    two_mores = 0
    
    for i in range(20,30):
        more_btn_path = '/html/body/table/tbody/tr/td/table[6]/tbody/tr[3]/td[2]/table/tbody/tr/td/table/tbody/tr/td/table[2]/tbody/tr/td/table[1]/tbody/tr['+ str(i)+']/td[1]/div[2]/a'
        #check if more button exists
        status = check_exists_by_xpath(more_btn_path, dr)
        if(status == 0):
            all_more.append(0)
        else:
            all_more.append(i)    
    if(max(all_more) == 0):
        return 0       
    else:
        if(all_more.count(0) == 7):
            target = second_largest(all_more)
            more_btn_path = '/html/body/table/tbody/tr/td/table[6]/tbody/tr[3]/td[2]/table/tbody/tr/td/table/tbody/tr/td/table[2]/tbody/tr/td/table[1]/tbody/tr['+ str(target)+']/td[1]/div[2]/a'
            #more_btn = driver.find_element_by_xpath(more_btn_path)
            more_btn = driver.find_element(By.XPATH, more_btn_path)
            return more_btn
        else:
            more_btn_path = '/html/body/table/tbody/tr/td/table[6]/tbody/tr[3]/td[2]/table/tbody/tr/td/table/tbody/tr/td/table[2]/tbody/tr/td/table[1]/tbody/tr['+ str(max(all_more))+']/td[1]/div[2]/a'
            #more_btn = driver.find_element_by_xpath(more_btn_path)
            more_btn = driver.find_element(By.XPATH, more_btn_path)
            return more_btn
        

def qc_status(page):
    passQC = page.find_all(string=re.compile("PassedQC"))
    failQC = page.find_all(string=re.compile("FailedQC"))
    pQC = len(passQC)
    fQC = len(failQC)
    if (pQC == 0 and fQC == 0):
        return 0, 0
    else:
        return round(pQC/(pQC+fQC), 2), round(fQC/(pQC+fQC), 2)

def gdv_checker(page, dr):
    gdv_btn_path = '//*[@id="gdv_button"]/span'
    status = check_exists_by_xpath(gdv_btn_path, dr)
    return status

def count_gene(page, gene, i):
    all_names = ""
    total = page.find_all(string=re.compile(str(gene[i])))
    for single in page.find_all(string=re.compile(str(gene[i]))):
        all_names = all_names + "|" + str(single)
    
    return all_names, len(total)


# Data Processing from Retrieved Results (Part 3)

The code below continues the data processing from the retrieved results. It retrieves additional information from the pages obtained earlier. Here's an overview of the code:

1. **Read the Excel File:** `workbook_df = pd.read_excel('test.xlsx')`
   - This line reads the previously saved Excel file (`test.xlsx`) into a DataFrame called `workbook_df`.

2. **Define Variables:**
   - `test = urls`
     - This line assigns the list of URLs (`urls`) to the variable `test`.

3. **Initialize Lists for Retrieved Information:**
   - `journal = []`, `organization = []`, `fQC = []`, `pQC = []`, `GDV = []`, `appearance = []`, `ref_num = []`
     - These lines initialize empty lists to store the retrieved information.

4. **Iterate Over URLs:**
   - The code then iterates over each URL in `test` using a for loop.

5. **Retrieve Information from Page:**
   - Inside the loop, it checks if the URL is not null using the `open_url(url, dr)` function. If the URL is not null, it proceeds with retrieving information from the page.

6. **Parse HTML and Retrieve Page Information:**
   - It retrieves the page source and creates a BeautifulSoup object (`page`) to parse the HTML.

7. **Check for "More" Button and Update Page:**
   - It checks for the existence of the "More" button using the `set_more(dr)` function. If the button exists, it clicks the button and retrieves the updated page source, creating a new BeautifulSoup object (`page`).

8. **Retrieve Journal Information:**
   - It retrieves the journal information using the `find_journal(page)` function and stores it in the `journal` list.

9. **Retrieve Organization Information:**
   - It retrieves the organization information using the `find_organization(page)` function and stores it in the `organization` list.

10. **Calculate QC Status:**
    - It calculates the QC status using the `qc_status(page)` function and stores the percentage of passed QC and failed QC in the `pQC` and `fQC` lists, respectively.

11. **Check for "gdv_button" and Store Result:**
    - It checks for the existence of the "gdv_button" using the `gdv_checker(page, dr)` function and stores the result in the `GDV` list.

12. **Count Gene Appearances on the Page:**
    - It counts the appearance of genes on the page using the `count_gene(page, genes, i)` function and stores the results in the `appearance` and `ref_num` lists.

13. **Handle Null URLs:**
    - If the URL is null, it appends NaN values to all the lists.

14. **Add Retrieved Information to DataFrame:**
    - After processing all URLs, it adds the retrieved information to the `workbook_df` DataFrame.

15. **Save Updated DataFrame to Excel File:**
    - Finally, it saves the updated DataFrame to a new Excel file called `test_final.xlsx`.

Please note that this Markdown cell provides a brief overview of the code. For more details on each function's implementation and usage, please refer to the code itself.


In [53]:
workbook_df =pd.read_excel('test.xlsx')

In [47]:
#test for searches
test = urls
#test = urls
driver = Chrome("chromedriver.exe")
journal = []
organization = []
fQC = []
pQC = []
GDV = []
appearance = []
ref_num = []





for i in range(0, len(test)):
    is_null = open_url(test[i], driver)
    
    if(is_null == 1):
        time.sleep(0.5)
        src = driver.page_source
        page = bs4.BeautifulSoup(src)
        
        more_btn = set_more(driver)
        if (more_btn != 0):
            more_btn.click()
            time.sleep(0.5)
            page = bs4.BeautifulSoup(src)
        
        jrl = find_journal(page) #pass
        org = find_organization(page) #pass  
        pass_QC, fail_QC = qc_status(page) #pass
        gdv_btn = gdv_checker(page, driver) #pass
        appear, refnum = count_gene(page, genes, i) #pass
        
        journal.append(jrl)
        organization.append(org)
        fQC.append(str(fail_QC))
        pQC.append(str(pass_QC))
        GDV.append(gdv_btn)
        appearance.append(appear)
        ref_num.append(refnum)
    
    else:
        journal.append(np.nan)
        organization.append(np.nan)
        fQC.append(np.nan)
        pQC.append(np.nan)
        GDV.append(np.nan)
        appearance.append(np.nan)
        ref_num.append(np.nan)
        
        
       

    
        
        
        
        
        
        
         
        


C:\Users\choyo\AppData\Local\Temp\ipykernel_12472\2295877218.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome("chromedriver.exe")


In [49]:
print(len(journal))
print(len(organization))
print(len(fQC))
print(len(pQC))
print(len(GDV))
print(len(appearance))
print(len(ref_num))


44
44
44
44
44
44
44


# Convert to dataframe for future analysis

In [50]:
workbook_df['Journal'] = journal
workbook_df['organization'] =organization
workbook_df['fQC'] =fQC
workbook_df['pQC'] =pQC
workbook_df['GDV'] =GDV
workbook_df['appearance'] =appearance
workbook_df['ref_num'] =ref_num
workbook_df.shape

In [51]:
#save
workbook_df.to_excel('test_final.xlsx', index = False, encoding = 'utf8')

(44, 10)